In [1]:
# Load Required Libraries
import os
import glob
import calendar

import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import  mapping #shape, 
import rasterio
# from rasterio.mask import mask
from rasterio.features import geometry_mask
# from contextlib import contextmanager

from osgeo import gdal #, osr, ogr

In [2]:
# Load Functions defined in functions4stepFunction jupyter notebook

%run functions4stepFunction.ipynb

In [3]:
# Define Annual and Pernnial crops

allcrops = {'Barley':'Bar', 'Cassava':'Cas', 'Cotton':'Cot', 'Fodder':'Fod', 'Groundnuts':'Gro',
            'Maize':'Maz', 'Millet':'Mil', 'Oil palm':'Olp', 'Others annual':'Otha', 'Others perennial':'Othp',
            'Potatoes':'Pot', 'Pulses':'Pul', 'Rapeseed':'Rap', 'Rice':'Ric', 'Rye':'Rye', 'Sorghum':'Sor',
            'Soybeans':'Soy', 'Coffee':'Stm', 'Cocoa':'Stm', 'Sugar beet':'Sub', 'Sugar cane':'Sug', 'Sunflower':'Sun', 
            'Wheat':'Whe'}
Annual = ("Barley", "Cassava", "Groundnuts","Maize","Millet",
          "Potatoes","Pulses","Rapeseed","Rice1","Rice2",
          "Sorghum","Soybeans","Sunflower","Wheat1","Wheat2",
          "Cotton", "Fodder", "Others annual", "Rye", "Sugar beet")

Perennial = ("Sugarcane", "Citrus", "Grape",  "Cocoa", "Coffee",
             "Oil palm", "Date palm", "Others perennial")


In [4]:
# Load the datasets
# ROOT FOLDER CONTAINS SHOULD CONTAIN CROP SUBFOLDERS WITH CLIPPED AREAS TIFF
root_folder = {y:"../data/Clipped_20{}".format(y) for y in ["00", "05", "10", "15"]}


# Load the crop and area data clipped by unit code 
raster_folder = "../data/Ras_shape_out/Rasters"
tiff_files = glob.glob("{}/**/*.tif".format(raster_folder), recursive=True)
tiff_files


# Folder with crop calendar data
CropCal_dir = "../data/CropCalender"

# Load crop calendar data and
# Rank each by crop area in each unit code
ranked_df_ir = {}
ranked_df_rf = {}
for year in ["00", "05", "10", "15"]:
  # Irrigated
  ranked_df_ir[year] = rankbyarea(CropCal_dir = CropCal_dir,
                                  irrigated_rainfed = "ir", year=year)
  # Rainfed
  ranked_df_rf[year] = rankbyarea(CropCal_dir = CropCal_dir,
                                  irrigated_rainfed = "rf", year=year)


# ranked_df_rf["15"]
ranked_df_rf["00"]

/tmp/ipykernel_879/31060390.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rank_crops) \
/tmp/ipykernel_879/31060390.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rank_crops) \


,rank,Country,State,District,unit_code,Crop,Subcrop,Type,Growing_area,Planting_Month,Maturity_Month
0,1,India,Andaman and Nicobar,Nicobar,3560001,Others annual2,1,Other Annual,35606,10,1
1,2,India,Andaman and Nicobar,Nicobar,3560001,Others annual1,1,Other Annual,622,6,10
2,3,India,Andaman and Nicobar,Nicobar,3560001,Barley,1,Annual,0,7,11
3,4,India,Andaman and Nicobar,Nicobar,3560001,Potatoes,1,Annual,0,6,10
4,5,India,Andaman and Nicobar,Nicobar,3560001,Wheat1,1,Annual,0,11,3
...,...,...,...,...,...,...,...,...,...,...,...
16830,22,India,West Bengal,Uttar Dinajpur,3560641,Sorghum,1,Annual,0,6,10
16831,23,India,West Bengal,Uttar Dinajpur,3560641,Sugar beet,1,Annual,0,6,10
16832,24,India,West Bengal,Uttar Dinajpur,3560641,Coffee,1,Perennial,0,1,12
16833,25,India,West Bengal,Uttar Dinajpur,3560641,Cocoa,1,Perennial,0,1,12


In [6]:


for year in ["00", "05", "10", "15"]:

  if ranked_df_ir[year] is None:
    print(11)
    # print("ranked_df_ir {} is not available".format(year))
    continue
  print(year)
  dist_array = {}
  irri_totals = {}

  # Get all unique UNIT CODEs
  basename_unit_codes = [os.path.splitext(os.path.basename(tiff_file))[0] \
                         for tiff_file in glob.glob("{}/*/**.tif".format(root_folder),
                                                    recursive=True)]
  unit_codes = np.unique(np.array(basename_unit_codes))

  for unit_code in unit_codes:
    print(unit_code)  

 
    # Get the crop-specific irrigated and rainfed growing area and crop calendar of the unit code district
    ranked_df_ir_unitcode = ranked_df_ir[year].loc[(ranked_df_ir[year]['unit_code'] == int(unit_code))]
    ranked_df_rf_unitcode = ranked_df_rf[year].loc[(ranked_df_rf[year]['unit_code'] == int(unit_code))]
    # print(ranked_df_unitcode)

    # SKIP UNIT CODES NOT IN IRRIGATED RANKED DATA
    if not ranked_df_ir_unitcode.shape[0]:
    # print("Unit code {} is not in ranked_df".format(unit_code))
      continue

    # LOAD THE AEI, ARA, AND CLE
    for aac in ["AEI", "Ara","CLE"]:
      tiff_file_aac = os.path.join("../{}".format(root_folder), "{}_{}".format(aac, year), "{}.tif".format(unit_code))
      # print(tiff_file_aac)
      dist_array["dist_array_{}".format(aac.lower())] = gdal.Open(tiff_file_aac, gdal.GA_ReadOnly).ReadAsArray()
 
     # Loop through the crops by rank while allocating area in the seven steps
    for j in range(1, int(np.max(ranked_df_ir_unitcode['rank']) + 1)):
      ranked_df_ir_rank = ranked_df_ir_unitcode.loc[(ranked_df_ir_unitcode['rank'] == j)]
      ranked_df_rf_rank = ranked_df_rf_unitcode.loc[(ranked_df_rf_unitcode['rank'] == j)]
      # print(ranked_df_rank)
      # Extract array data for crop type and assign to corresponding variable in crop_dict
      crop_name = ranked_df_ir_rank.Crop.tolist()[0]


      # GET the TIFF FILE FOR THE CROP
      tiff_file = None
      for k in allcrops.keys():
        if k in crop_name:
          tiff_file = os.path.join("../{}".format(root_folder), 
                                 "{}_{}".format(allcrops[k], year), 
                                 "{}.tif".format(unit_code))
          # print("FOUND")
          break
      # SKIP THE ROW IF CROP NOT IN ALL CROPS
      if not tiff_file:
        # print("Crop {} is not in allcrops dictionary".format(crop_name))
        break
      # print(tiff_file)

    
      AH_array = gdal.Open(tiff_file, gdal.GA_ReadOnly).ReadAsArray()
      AH_array[AH_array < 0] = 0  

      
      # CALL THE STEPS ONE AT A TIME WITH THE DATA  AND FIX THE ERRORS
      step1 = step00(step=1, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = None, ATS_2 = None, AD_area = None, df_cumulative=None,
                     df_mon=None, TAD_12=None)
  
      # {"df_cumulative":df_cumulative, "AD_area" : AD_area1, "ATS_2": ATS_2, "df_tot":df_tot}
      step2 = step00(step=2, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank, 
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step1['df_tot'],  ATS_2 = step1['ATS_2'], AD_area = step1['AD_area'],
                     df_cumulative=step1['df_cumulative'], df_mon=step1['df_mon'],
                     c_s=step1['c_s'], r_s=step1['r_s'],
                     TAD_12=None)
    
      step3 = step00(step=3, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step2['df_tot'],  ATS_2 = step2['ATS_2'], AD_area = step2['AD_area'], 
                     df_cumulative=step2['df_cumulative'], df_mon=step2['df_mon'],
                     c_s=step2['c_s'], r_s=step2['r_s'],
                     TAD_12=None)


    # cHECK IF THE CROP IN BOTH IRRIGATED AND RAINFED
    
      step4 = step00(step=4, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step3['df_tot'],  ATS_2 = step3['ATS_2'], AD_area = step3['AD_area'],
                     df_cumulative=step3['df_cumulative'], df_mon=step3['df_mon'],
                     c_s=step3['c_s'], r_s=step3['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)

      # if dist_array_area is not None and dist_array_aei is not None and dist_array_cle is not None:
      array0 = [dist_array["dist_array_ara"], dist_array["dist_array_aei"], dist_array["dist_array_cle"]]
      if any(element is None for element in array0):
        break
      step5 = step00(step=5, year = year, ranked_df_ir_rank = ranked_df_ir_rank, 
                     ranked_df_rf_rank = ranked_df_rf_rank,
                     dist_array = dist_array, AH_array = AH_array,
                     df_tot = step4['df_tot'],  ATS_2 = step4['ATS_2'], AD_area = step4['AD_area'],
                     df_cumulative=step4['df_cumulative'], df_mon=step4['df_mon'],
                     c_s=step4['c_s'], r_s=step4['r_s'],
                     TAD_12=None,
                     template_tiff= tiff_file)

  # print(step5)


00
11
11
11


TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union